In [0]:
# ==============================================================================
# CONFIGURAÇÕES E IMPORTS
# ==============================================================================
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Configurar o uso do nosso catálogo e dos schemas
spark.sql("USE CATALOG previsao_brasileirao")

# Definir os schemas de origem (Bronze) e destino (Silver)
bronze_schema = "bronze"
silver_schema = "silver"

print(f"Lendo dados de: {bronze_schema}")
print(f"Salvando dados em: {silver_schema}")

In [0]:
# ==============================================================================
# PASSO 1: LIMPAR E ESTRUTURAR A TABELA DE PARTIDAS
# ==============================================================================
print("Iniciando processamento da tabela de partidas...")

# Ler a tabela bruta da camada Bronze
df_partidas_bronze = spark.table(f"{bronze_schema}.partidas_raw")

df_partidas_silver = (
    df_partidas_bronze
    # Renomear colunas para um padrão mais claro
    .withColumnRenamed("clube_casa_id", "mandante_id")
    .withColumnRenamed("clube_visitante_id", "visitante_id")
    .withColumnRenamed("placar_oficial_mandante", "mandante_placar")
    .withColumnRenamed("placar_oficial_visitante", "visitante_placar")
    .withColumnRenamed("partida_data", "data_partida")
    
    # Converter a data da partida para o formato Timestamp
    .withColumn("data_partida", F.to_timestamp("data_partida"))
    
    # Criar a variável alvo (target) para o modelo de classificação
    .withColumn("resultado",
        F.when(F.col("mandante_placar") > F.col("visitante_placar"), "VITORIA_MANDANTE")
         .when(F.col("mandante_placar") < F.col("visitante_placar"), "VITORIA_VISITANTE")
         .when(F.col("mandante_placar") == F.col("visitante_placar"), "EMPATE")
         .otherwise(None) # Para partidas futuras que não têm placar
    )
    # Selecionar apenas as colunas relevantes para a camada Silver
    .select(
        "partida_id",
        "rodada",
        "data_partida",
        "mandante_id",
        "visitante_id",
        "mandante_placar",
        "visitante_placar",
        "resultado",
        "valida"
    )
    # Filtrar apenas as partidas válidas que já aconteceram (têm resultado)
    .filter("resultado IS NOT NULL AND valida = true")
)

# Salvar a tabela limpa na camada Silver
df_partidas_silver.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_schema}.partidas")

print(f"✅ Tabela 'partidas' salva com sucesso na camada Silver.")
display(spark.table(f"{silver_schema}.partidas").orderBy(F.desc("data_partida")).limit(10))

In [0]:
# ==============================================================================
# PASSO 2: DESEMPACOTAR SCOUTS E CRIAR ESTATÍSTICAS POR JOGADOR
# ==============================================================================
print("Iniciando processamento da tabela de pontuações e scouts...")

# Ler a tabela bruta
df_pontuacoes_bronze = spark.table(f"{bronze_schema}.pontuacoes_historico_raw")

# --- AJUSTE APLICADO AQUI ---
# A coluna 'scout' já é uma estrutura (MapType ou StructType), não uma string JSON.
# Em vez de usar from_json, vamos selecionar os elementos diretamente.

# Lista de todos os scouts possíveis para selecionar
lista_de_scouts = ["A", "CA", "CV", "DD", "DP", "DS", "FC", "FD", "FF", "FS", "G", "GC", "GS", "I", "PE", "RB", "SG"]
# Criar a lista de colunas para o select dinamicamente
select_scouts = [F.col(f"scout.{scout}").alias(scout) for scout in lista_de_scouts]

df_stats_silver = (
    df_pontuacoes_bronze
    .select(
        "atleta_id",
        "rodada_id",
        "clube_id",
        "pontos_num",
        *select_scouts # O asterisco desempacota a lista de colunas aqui
    )
    # Substituir quaisquer valores nulos nos scouts (para jogadores que não tiveram o scout) por 0
    .na.fill(0, subset=lista_de_scouts)
)
# --- FIM DO AJUSTE ---

# Juntar com a tabela de partidas para obter o partida_id
df_partidas_silver_ref = spark.table(f"{silver_schema}.partidas").select("rodada", "mandante_id", "visitante_id", "partida_id")

# Criar duas visões das partidas para juntar mandantes e visitantes
partidas_mandante = df_partidas_silver_ref.select(F.col("rodada"), F.col("mandante_id").alias("clube_id"), F.col("partida_id"))
partidas_visitante = df_partidas_silver_ref.select(F.col("rodada"), F.col("visitante_id").alias("clube_id"), F.col("partida_id"))
partidas_clubes = partidas_mandante.union(partidas_visitante)

# Juntar as estatísticas com as partidas para ter o contexto completo
# Adicionado alias para evitar colunas ambíguas no join
df_stats_silver_aliased = df_stats_silver.alias("stats")
partidas_clubes_aliased = partidas_clubes.alias("partidas")

df_estatisticas_final = df_stats_silver_aliased.join(
    partidas_clubes_aliased,
    (F.col("stats.rodada_id") == F.col("partidas.rodada")) & (F.col("stats.clube_id") == F.col("partidas.clube_id")),
    "inner"
).select(
    "partidas.partida_id",
    "stats.rodada_id",
    "stats.atleta_id",
    "stats.clube_id",
    "stats.pontos_num",
    *lista_de_scouts # Reutiliza a lista para selecionar todas as colunas de scout
)


# Salvar a tabela final na camada Silver
df_estatisticas_final.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_schema}.estatisticas_jogador_partida")

print(f"✅ Tabela 'estatisticas_jogador_partida' salva com sucesso na camada Silver.")
display(spark.table(f"{silver_schema}.estatisticas_jogador_partida").limit(10))

In [0]:
# ==============================================================================
# PASSO 3: LIMPAR E PADRONIZAR A TABELA DE CLUBES
# ==============================================================================
print("Iniciando processamento da tabela de clubes...")

df_clubes_silver = (
    spark.table(f"{bronze_schema}.clubes_info_raw")
    .withColumnRenamed("id", "clube_id")
    .withColumnRenamed("nome_fantasia", "nome_completo")
    .select("clube_id", "nome", "abreviacao", "nome_completo")
)

df_clubes_silver.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable(f"{silver_schema}.clubes")

print(f"✅ Tabela 'clubes' salva com sucesso na camada Silver.")
display(spark.table(f"{silver_schema}.clubes"))

In [0]:
# ==============================================================================
# VALIDAÇÃO DA CAMADA SILVER
# ==============================================================================
print("--- Sumário da Camada Silver ---")
for table_name in ["partidas", "estatisticas_jogador_partida", "clubes"]:
    print(f"\nTabela: {silver_schema}.{table_name}")
    df = spark.table(f"{silver_schema}.{table_name}")
    print(f"  - Registros: {df.count():,}")
    print(f"  - Colunas: {len(df.columns)}")
    df.printSchema()